In [20]:
import os
import numpy as np
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset

dataset_path = '/media/n31v/data/datasets/minerals'

transform = Compose([Resize((1920, 1080)), ToTensor()])
dataset = ImageFolder(root=dataset_path, transform=transform)
dataset = Subset(dataset, indices=np.load(os.path.join(dataset_path, 'balanced.npy')))
val_ds = Subset(dataset, indices=np.load(os.path.join(dataset_path, 'val.npy')))
dl = DataLoader(val_ds)

In [1]:
from torchvision.models import resnet50
from core.architecture.experiment.nn_experimenter import ClassificationExperimenter
from utils import profile, flop

experimenter = ClassificationExperimenter(
    model=resnet50(num_classes=7),
    name='ResNet50_lrs'
)
print(experimenter.size_of_model())
flop(profile(experimenter.model, 'base'))

94.302404


STAGE:2023-03-07 14:39:20 1387071:1387071 ActivityProfilerController.cpp:294] Completed Stage: Warm Up


aten::conv2d: 339143393280 flops, 53 calls
aten::addmm: 28672 flops, 1 calls


STAGE:2023-03-07 14:39:21 1387071:1387071 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-03-07 14:39:21 1387071:1387071 output_json.cpp:417] Completed Stage: Post Processing


339143421952

In [2]:
from core.operation.optimization.structure_optimization import SVDOptimization

svd_optim = SVDOptimization(
    energy_thresholds=[0.1, 0.3, 0.5, 0.7, 0.9, 0.93, 0.96, 0.99, 0.999],
    decomposing_mode='spatial',
    hoer_loss_factor=0.01
)
svd_optim.load_model(experimenter, state_dict_path='models/minerals(1920х1080)/ResNet50_lrs_SVD_spatial_O-100.0_H-0.001000/e_0.9.sd.pt')
print(experimenter.size_of_model())
flop(profile(experimenter.model, 'svd'))
# experimenter.val_loop(dataloader=dl)

2023-03-07 14:40:19,513 - SVDOptimization - Model state dict loaded.
93.050416
aten::mm: 35033306752 flops, 106 calls
aten::conv2d: 339143393280 flops, 53 calls
aten::addmm: 28672 flops, 1 calls


STAGE:2023-03-07 14:40:19 1387071:1387071 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-07 14:40:19 1387071:1387071 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-03-07 14:40:19 1387071:1387071 output_json.cpp:417] Completed Stage: Post Processing


374176728704

aten::mm: 35033306752
aten::conv2d: 339143393280
aten::addmm: 28672
total: 374176728704

In [3]:
from core.operation.decomposition.decomposed_conv import DecomposedConv2d

def compose_layer(conv: DecomposedConv2d):
    conv.compose()

experimenter.apply_func(func=compose_layer, condition=lambda x: isinstance(x, DecomposedConv2d))
print(experimenter.size_of_model())
flop(profile(experimenter.model, 'svd compose'))
# experimenter.val_loop(dataloader=dl)

94.302404


STAGE:2023-03-07 14:40:19 1387071:1387071 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-07 14:40:19 1387071:1387071 ActivityProfilerController.cpp:300] Completed Stage: Collection


aten::conv2d: 339143393280 flops, 53 calls
aten::addmm: 28672 flops, 1 calls


STAGE:2023-03-07 14:40:19 1387071:1387071 output_json.cpp:417] Completed Stage: Post Processing


339143421952

{'accuracy': 0.707641196013289,
 'precision': 0.7376112940207636,
 'recall': 0.707641196013289,
 'f1': 0.7062899526066728,
 'roc_auc': 0.9052898606711478}

aten::conv2d: 339143393280
aten::addmm: 28672
total: 339143421952

In [4]:
import copy
from core.operation.optimization.sfp_tools import create_energy_filter_zeroing_fn, create_percentage_filter_zeroing_fn, prune_resnet
filter_zeroing = create_energy_filter_zeroing_fn(0.96)
# filter_zeroing = create_percentage_filter_zeroing_fn(0.1)

experimenter.apply_func(func=filter_zeroing, condition=lambda x: isinstance(x, DecomposedConv2d))
experimenter.model = prune_resnet(model=experimenter.model)
experimenter.model.to(experimenter.device)
print(experimenter.size_of_model())
flop(profile(experimenter.model, 'svd pruned'))

88.150908
aten::conv2d: 315082416480 flops, 53 calls
aten::addmm: 28672 flops, 1 calls


STAGE:2023-03-07 14:40:41 1387071:1387071 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-07 14:40:41 1387071:1387071 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-03-07 14:40:41 1387071:1387071 output_json.cpp:417] Completed Stage: Post Processing


315082445152

In [21]:
experimenter.val_loop(dataloader=dl)

100%|██████████| 301/301 [00:57<00:00,  5.23it/s]


{'accuracy': 0.8006644518272426,
 'precision': 0.8176867752339451,
 'recall': 0.8006644518272426,
 'f1': 0.8014518581409948,
 'roc_auc': 0.9188235597105258}

In [8]:
from torchvision.models import resnet50

from core.architecture.experiment.nn_experimenter import ClassificationExperimenter

experimenter = ClassificationExperimenter(
    model=resnet50(num_classes=7),
    name='ResNet50_lrs',
    weights='models/minerals(1920х1080)/ResNet50_lrs_SFP_energy_threshold-0.994/train.sd.pt'
)
print(experimenter.size_of_model())
flop(profile(experimenter.model, 'sfp'))

94.302404


STAGE:2023-03-07 14:43:53 1387071:1387071 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-07 14:43:53 1387071:1387071 ActivityProfilerController.cpp:300] Completed Stage: Collection


aten::conv2d: 339143393280 flops, 53 calls
aten::addmm: 28672 flops, 1 calls


STAGE:2023-03-07 14:43:54 1387071:1387071 output_json.cpp:417] Completed Stage: Post Processing


339143421952

In [9]:
from core.operation.optimization.sfp_tools import prune_resnet
from utils import convert_ONNX

experimenter.model = prune_resnet(experimenter.model, mk=True)
print(experimenter.size_of_model())
print(flop(profile(experimenter.model, 'sfp_mk')))
convert_ONNX(experimenter.model, 'sfp')

42.832716
aten::conv2d: 160247079120 flops, 53 calls
aten::addmm: 28672 flops, 1 calls
160247107792


STAGE:2023-03-07 14:43:55 1387071:1387071 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-07 14:43:55 1387071:1387071 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-03-07 14:43:55 1387071:1387071 output_json.cpp:417] Completed Stage: Post Processing


Exported graph: graph(%input.1 : Float(1, 3, 1920, 1080, strides=[6220800, 2073600, 1080, 1], requires_grad=1, device=cuda:0),
      %fc.weight : Float(7, 2048, strides=[2048, 1], requires_grad=1, device=cuda:0),
      %fc.bias : Float(7, strides=[1], requires_grad=1, device=cuda:0),
      %onnx::Conv_497 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_498 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_500 : Float(17, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_501 : Float(17, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_503 : Float(29, 17, 3, 3, strides=[153, 9, 3, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_504 : Float(29, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_506 : Float(256, 29, 1, 1, strides=[29, 1, 1, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_507 : Float(256, strides=[1], requires_grad=0, device=

KeyboardInterrupt: 

In [ ]:
experimenter.val_loop(dataloader=dl)

In [17]:
from torchvision.models import resnet50

from core.architecture.experiment.nn_experimenter import ClassificationExperimenter
from core.operation.optimization.structure_optimization import SFPOptimization

experimenter = ClassificationExperimenter(
    model=resnet50(num_classes=7),
    name='ResNet50_lrs'
)
sfp_optim = SFPOptimization(zeroing_mode='energy', zeroing_mode_params={'energy_threshold': 0.994})
sfp_optim.load_model(exp=experimenter, state_dict_path='models/minerals(1920х1080)/ResNet50_lrs_SFP_energy_threshold-0.994/pruned.sd.pt')
print(experimenter.size_of_model())
flop(profile(experimenter.model, 'sfp_pruned'))

2023-03-07 13:37:35,102 - SFPOptimization - Model loaded.
32.50124
aten::conv2d: 126824387280 flops, 53 calls
aten::addmm: 21252 flops, 1 calls
total: 126824408532


STAGE:2023-03-07 13:37:35 1262509:1262509 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-07 13:37:35 1262509:1262509 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-03-07 13:37:35 1262509:1262509 output_json.cpp:417] Completed Stage: Post Processing
